## Understanding the data


FSP entries : 10630

Backend entries : 10758

Difference of 128

FSP matched = 5309
    unmatched = 5321
    
Backend matched = 10723
        unmatched = 35 

What is the matching reference ?
for backend : clientId @ Number contained in description , Amount
for frontend : clientId @ Number after tp cosmic in description, backend matching reference

Taking example row 1090 fsp and row 6092 backend

In backend, the unmatched entries do not have an fsp transaction id - this col may be filled after matching therefore can not be used in fuzzy matching.

Amount is held constant, so it may be used to create the matches.

Where the description is alphanumeric as opposed to numeric, the entry remains unmatched.

Assuming backend is recorded first.



In [14]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import re

nomba_gtb_fsp = pd.read_csv('Nomba_GTB_BILLS - fsp_16_mar_2024_31_mar_2024.csv')
nomba_gtb_backend = pd.read_csv('Nomba_GTB_BILLS - Backend_1_Mar_2024_31_2024.csv')


In [15]:
nomba_gtb_backend

,id_backend,transaction_date_backend,value_date,transaction_type,amount_backend,amount_currency,balance,balance_currency,matching_reference,reporting_tag,...,updated,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description
0,684754662,2024-03-31,2024-03-31 21:21:30.000000 UTC,C,2950,NGN,0,NGN,[3@39188343199314622950.00],transfers,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 21:21:30, origina...",[transfers],3918834319931462/SUCCESS
1,684754663,2024-03-31,2024-03-31 20:20:30.000000 UTC,C,350,NGN,0,NGN,[3@2347647791697967350.00],transfers,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 20:20:30, origina...",[transfers],2347647791697967/SUCCESS
2,684754665,2024-03-31,2024-03-31 23:14:38.000000 UTC,C,1950,NGN,0,NGN,[3@38925732955313971950.00],transfers,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 23:14:38, origina...",[transfers],3892573295531397/SUCCESS
3,684754666,2024-03-31,2024-03-31 21:03:38.000000 UTC,C,1700,NGN,0,NGN,[3@72433543269619881700.00],transfers,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 21:03:38, origina...",[transfers],7243354326961988/SUCCESS
4,684754668,2024-03-31,2024-03-31 22:00:07.000000 UTC,C,3950,NGN,0,NGN,[3@14616371572939783950.00],transfers,...,2024-04-12 01:52:53.248606 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 22:00:07, origina...",[transfers],1461637157293978/SUCCESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10753,684756326,2024-03-03,2024-03-03 18:14:09.000000 UTC,C,18000,NGN,0,NGN,[3@857975845556732518000.00],transfers,...,2024-05-24 00:17:49.000000 UTC,[70],[489667289],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 18:14:09, origina...",[transfers],8579758455567325/SUCCESS
10754,684758495,2024-03-03,2024-03-03 16:04:26.000000 UTC,C,500,NGN,0,NGN,[3@2921958749136149500.00],transfers,...,2024-05-24 00:17:49.000000 UTC,[70],[489667203],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 16:04:26, origina...",[transfers],2921958749136149/SUCCESS
10755,684756787,2024-03-05,2024-03-05 10:52:03.000000 UTC,C,200,NGN,0,NGN,[3@1638195999657187200.00],transfers,...,2024-05-24 00:17:49.000000 UTC,[70],[490523094],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-05 10:52:03, origina...",[transfers],1638195999657187/SUCCESS
10756,684763770,2024-03-04,2024-03-04 17:27:11.000000 UTC,C,10000,NGN,0,NGN,[3@134952516175486110000.00],transfers,...,2024-05-24 00:17:49.000000 UTC,[70],[489667871],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-04 17:27:11, origina...",[transfers],1349525161754861/SUCCESS


In [16]:
nomba_gtb_fsp

,id_fsp,transaction_date_fsp,value_date,transaction_type,amount_fsp,amount_currency,balance,balance_currency,matching_reference,reporting_tag,...,updated,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description
0,493527019,2024-03-24,2024-03-24 00:00:00.000000 UTC,D,400.00,NGN,23387956.15,NGN,"[3@3622512813388183, 3@3622512813388183400.00]",bill_payments,...,2024-05-16 15:46:34.000000 UTC,NaN,NaN,[136],[684757099],NaN,NaN,"{transaction_date=2024-03-24 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...
1,493530399,2024-03-29,2024-03-29 00:00:00.000000 UTC,D,100.00,NGN,22723019.15,NGN,"[3@4122389416478862100.00, 3@4122389416478862]",bill_payments,...,2024-05-16 15:46:34.000000 UTC,NaN,NaN,[136],[684756545],NaN,NaN,"{transaction_date=2024-03-29 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...
2,493531321,2024-03-30,2024-03-30 00:00:00.000000 UTC,D,1000.00,NGN,17587584.15,NGN,"[3@1332568146768148, 3@13325681467681481000.00]",bill_payments,...,2024-05-16 15:46:34.000000 UTC,NaN,NaN,[136],[684759010],NaN,NaN,"{transaction_date=2024-03-30 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...
3,493523573,2024-03-17,2024-03-17 00:00:00.000000 UTC,D,1000.00,NGN,22078736.15,NGN,"[3@7321192357487326, 3@73211923574873261000.00]",bill_payments,...,2024-05-16 15:46:34.000000 UTC,NaN,NaN,[136],[684760491],NaN,NaN,"{transaction_date=2024-03-17 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...
4,493526877,2024-03-23,2024-03-23 00:00:00.000000 UTC,D,500.00,NGN,24417139.15,NGN,"[3@2291774841227371500.00, 3@2291774841227371]",bill_payments,...,2024-05-16 15:46:34.000000 UTC,NaN,NaN,[136],[684758199],NaN,NaN,"{transaction_date=2024-03-23 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10625,493532512,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12739233.15,NGN,"[3@2729785645691423, 3@272978564569142325.00]",fees_and_commissions,...,2024-05-16 15:45:31.230716 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...
10626,493532514,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12724208.15,NGN,"[3@842262142923286825.00, 3@8422621429232868]",fees_and_commissions,...,2024-05-16 15:45:31.230716 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...
10627,493532516,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12709183.15,NGN,"[3@369436493293619825.00, 3@3694364932936198]",fees_and_commissions,...,2024-05-16 15:45:31.230716 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...
10628,493532517,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25320.70,NGN,12683862.45,NGN,NaN,fees_and_commissions,...,2024-05-16 15:45:31.230716 UTC,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...","[no_ref, fees_and_commissions]",D | ACCOUNT MAINTENANCE CHARGE


Extracting the number from description in backend

In [17]:
nomba_gtb_backend['description_number'] = nomba_gtb_backend['description'].str.split('/').str[0]

In [18]:
nomba_gtb_backend

,id_backend,transaction_date_backend,value_date,transaction_type,amount_backend,amount_currency,balance,balance_currency,matching_reference,reporting_tag,...,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description,description_number
0,684754662,2024-03-31,2024-03-31 21:21:30.000000 UTC,C,2950,NGN,0,NGN,[3@39188343199314622950.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 21:21:30, origina...",[transfers],3918834319931462/SUCCESS,3918834319931462
1,684754663,2024-03-31,2024-03-31 20:20:30.000000 UTC,C,350,NGN,0,NGN,[3@2347647791697967350.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 20:20:30, origina...",[transfers],2347647791697967/SUCCESS,2347647791697967
2,684754665,2024-03-31,2024-03-31 23:14:38.000000 UTC,C,1950,NGN,0,NGN,[3@38925732955313971950.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 23:14:38, origina...",[transfers],3892573295531397/SUCCESS,3892573295531397
3,684754666,2024-03-31,2024-03-31 21:03:38.000000 UTC,C,1700,NGN,0,NGN,[3@72433543269619881700.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 21:03:38, origina...",[transfers],7243354326961988/SUCCESS,7243354326961988
4,684754668,2024-03-31,2024-03-31 22:00:07.000000 UTC,C,3950,NGN,0,NGN,[3@14616371572939783950.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 22:00:07, origina...",[transfers],1461637157293978/SUCCESS,1461637157293978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10753,684756326,2024-03-03,2024-03-03 18:14:09.000000 UTC,C,18000,NGN,0,NGN,[3@857975845556732518000.00],transfers,...,[70],[489667289],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 18:14:09, origina...",[transfers],8579758455567325/SUCCESS,8579758455567325
10754,684758495,2024-03-03,2024-03-03 16:04:26.000000 UTC,C,500,NGN,0,NGN,[3@2921958749136149500.00],transfers,...,[70],[489667203],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 16:04:26, origina...",[transfers],2921958749136149/SUCCESS,2921958749136149
10755,684756787,2024-03-05,2024-03-05 10:52:03.000000 UTC,C,200,NGN,0,NGN,[3@1638195999657187200.00],transfers,...,[70],[490523094],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-05 10:52:03, origina...",[transfers],1638195999657187/SUCCESS,1638195999657187
10756,684763770,2024-03-04,2024-03-04 17:27:11.000000 UTC,C,10000,NGN,0,NGN,[3@134952516175486110000.00],transfers,...,[70],[489667871],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-04 17:27:11, origina...",[transfers],1349525161754861/SUCCESS,1349525161754861


Extracting the number from description in fsp using the import re

In [23]:
def extract_fsp_description_number(text): 
    match = re.search(r'TP-COSMIC\s+(\d+)',text)
    return match.group(1) if match else None

nomba_gtb_fsp['description_number_fsp'] = nomba_gtb_fsp['description'].apply(extract_fsp_description_number)

In [24]:
nomba_gtb_fsp

,id_fsp,transaction_date_fsp,value_date,transaction_type,amount_fsp,amount_currency,balance,balance_currency,matching_reference,reporting_tag,...,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description,description_number_fsp
0,493527019,2024-03-24,2024-03-24 00:00:00.000000 UTC,D,400.00,NGN,23387956.15,NGN,"[3@3622512813388183, 3@3622512813388183400.00]",bill_payments,...,NaN,NaN,[136],[684757099],NaN,NaN,"{transaction_date=2024-03-24 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,3622512813388183
1,493530399,2024-03-29,2024-03-29 00:00:00.000000 UTC,D,100.00,NGN,22723019.15,NGN,"[3@4122389416478862100.00, 3@4122389416478862]",bill_payments,...,NaN,NaN,[136],[684756545],NaN,NaN,"{transaction_date=2024-03-29 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,4122389416478862
2,493531321,2024-03-30,2024-03-30 00:00:00.000000 UTC,D,1000.00,NGN,17587584.15,NGN,"[3@1332568146768148, 3@13325681467681481000.00]",bill_payments,...,NaN,NaN,[136],[684759010],NaN,NaN,"{transaction_date=2024-03-30 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,1332568146768148
3,493523573,2024-03-17,2024-03-17 00:00:00.000000 UTC,D,1000.00,NGN,22078736.15,NGN,"[3@7321192357487326, 3@73211923574873261000.00]",bill_payments,...,NaN,NaN,[136],[684760491],NaN,NaN,"{transaction_date=2024-03-17 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,7321192357487326
4,493526877,2024-03-23,2024-03-23 00:00:00.000000 UTC,D,500.00,NGN,24417139.15,NGN,"[3@2291774841227371500.00, 3@2291774841227371]",bill_payments,...,NaN,NaN,[136],[684758199],NaN,NaN,"{transaction_date=2024-03-23 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,2291774841227371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10625,493532512,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12739233.15,NGN,"[3@2729785645691423, 3@272978564569142325.00]",fees_and_commissions,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...,2729785645691423
10626,493532514,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12724208.15,NGN,"[3@842262142923286825.00, 3@8422621429232868]",fees_and_commissions,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...,8422621429232868
10627,493532516,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12709183.15,NGN,"[3@369436493293619825.00, 3@3694364932936198]",fees_and_commissions,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...,3694364932936198
10628,493532517,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25320.70,NGN,12683862.45,NGN,NaN,fees_and_commissions,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...","[no_ref, fees_and_commissions]",D | ACCOUNT MAINTENANCE CHARGE,None


Reading amount from backend and fsp into the same format (2 decimal places)

In [31]:
nomba_gtb_backend['amount_backend'] = nomba_gtb_backend['amount_backend'].astype(float).map('{:.2f}'.format)
nomba_gtb_fsp['amount_fsp'] = nomba_gtb_fsp['amount_fsp'].astype(float).map('{:.2f}'.format)

nomba_gtb_fsp


,id_fsp,transaction_date_fsp,value_date,transaction_type,amount_fsp,amount_currency,balance,balance_currency,matching_reference,reporting_tag,...,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description,description_number_fsp
0,493527019,2024-03-24,2024-03-24 00:00:00.000000 UTC,D,400.00,NGN,23387956.15,NGN,"[3@3622512813388183, 3@3622512813388183400.00]",bill_payments,...,NaN,NaN,[136],[684757099],NaN,NaN,"{transaction_date=2024-03-24 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,3622512813388183
1,493530399,2024-03-29,2024-03-29 00:00:00.000000 UTC,D,100.00,NGN,22723019.15,NGN,"[3@4122389416478862100.00, 3@4122389416478862]",bill_payments,...,NaN,NaN,[136],[684756545],NaN,NaN,"{transaction_date=2024-03-29 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,4122389416478862
2,493531321,2024-03-30,2024-03-30 00:00:00.000000 UTC,D,1000.00,NGN,17587584.15,NGN,"[3@1332568146768148, 3@13325681467681481000.00]",bill_payments,...,NaN,NaN,[136],[684759010],NaN,NaN,"{transaction_date=2024-03-30 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,1332568146768148
3,493523573,2024-03-17,2024-03-17 00:00:00.000000 UTC,D,1000.00,NGN,22078736.15,NGN,"[3@7321192357487326, 3@73211923574873261000.00]",bill_payments,...,NaN,NaN,[136],[684760491],NaN,NaN,"{transaction_date=2024-03-17 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,7321192357487326
4,493526877,2024-03-23,2024-03-23 00:00:00.000000 UTC,D,500.00,NGN,24417139.15,NGN,"[3@2291774841227371500.00, 3@2291774841227371]",bill_payments,...,NaN,NaN,[136],[684758199],NaN,NaN,"{transaction_date=2024-03-23 00:00:00, origina...",[bill_payments],D | TRANSFER BETWEEN CUSTOMERS BillsPayment Pa...,2291774841227371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10625,493532512,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12739233.15,NGN,"[3@2729785645691423, 3@272978564569142325.00]",fees_and_commissions,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...,2729785645691423
10626,493532514,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12724208.15,NGN,"[3@842262142923286825.00, 3@8422621429232868]",fees_and_commissions,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...,8422621429232868
10627,493532516,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12709183.15,NGN,"[3@369436493293619825.00, 3@3694364932936198]",fees_and_commissions,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...",[fees_and_commissions],D | COMMISSION BillsPayment Payment Charge1 vi...,3694364932936198
10628,493532517,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25320.70,NGN,12683862.45,NGN,NaN,fees_and_commissions,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 00:00:00, origina...","[no_ref, fees_and_commissions]",D | ACCOUNT MAINTENANCE CHARGE,None


In [32]:
nomba_gtb_backend

,id_backend,transaction_date_backend,value_date,transaction_type,amount_backend,amount_currency,balance,balance_currency,matching_reference,reporting_tag,...,fsp_accounts,fsp_transaction_ids,backend_accounts,backend_transaction_ids,other_accounts,other_transaction_ids,metadata,tags,description,description_number
0,684754662,2024-03-31,2024-03-31 21:21:30.000000 UTC,C,2950.00,NGN,0,NGN,[3@39188343199314622950.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 21:21:30, origina...",[transfers],3918834319931462/SUCCESS,3918834319931462
1,684754663,2024-03-31,2024-03-31 20:20:30.000000 UTC,C,350.00,NGN,0,NGN,[3@2347647791697967350.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 20:20:30, origina...",[transfers],2347647791697967/SUCCESS,2347647791697967
2,684754665,2024-03-31,2024-03-31 23:14:38.000000 UTC,C,1950.00,NGN,0,NGN,[3@38925732955313971950.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 23:14:38, origina...",[transfers],3892573295531397/SUCCESS,3892573295531397
3,684754666,2024-03-31,2024-03-31 21:03:38.000000 UTC,C,1700.00,NGN,0,NGN,[3@72433543269619881700.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 21:03:38, origina...",[transfers],7243354326961988/SUCCESS,7243354326961988
4,684754668,2024-03-31,2024-03-31 22:00:07.000000 UTC,C,3950.00,NGN,0,NGN,[3@14616371572939783950.00],transfers,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 22:00:07, origina...",[transfers],1461637157293978/SUCCESS,1461637157293978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10753,684756326,2024-03-03,2024-03-03 18:14:09.000000 UTC,C,18000.00,NGN,0,NGN,[3@857975845556732518000.00],transfers,...,[70],[489667289],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 18:14:09, origina...",[transfers],8579758455567325/SUCCESS,8579758455567325
10754,684758495,2024-03-03,2024-03-03 16:04:26.000000 UTC,C,500.00,NGN,0,NGN,[3@2921958749136149500.00],transfers,...,[70],[489667203],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-03 16:04:26, origina...",[transfers],2921958749136149/SUCCESS,2921958749136149
10755,684756787,2024-03-05,2024-03-05 10:52:03.000000 UTC,C,200.00,NGN,0,NGN,[3@1638195999657187200.00],transfers,...,[70],[490523094],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-05 10:52:03, origina...",[transfers],1638195999657187/SUCCESS,1638195999657187
10756,684763770,2024-03-04,2024-03-04 17:27:11.000000 UTC,C,10000.00,NGN,0,NGN,[3@134952516175486110000.00],transfers,...,[70],[489667871],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-04 17:27:11, origina...",[transfers],1349525161754861/SUCCESS,1349525161754861


In [33]:
left_on =["description_number_fsp","amount_fsp","transaction_date_fsp"]
right_on = ["description_number","amount_backend","transaction_date_backend"]

In [34]:
matched_results = fuzzymatcher.fuzzy_left_join(nomba_gtb_fsp, nomba_gtb_backend, left_on, right_on, left_id_col="id_fsp", right_id_col="id_backend")

In [35]:
matched_results 

,best_match_score,__id_left,__id_right,id_fsp,transaction_date_fsp,value_date_left,transaction_type_left,amount_fsp,amount_currency_left,balance_left,...,fsp_accounts_right,fsp_transaction_ids_right,backend_accounts_right,backend_transaction_ids_right,other_accounts_right,other_transaction_ids_right,metadata_right,tags_right,description_right,description_number
0,0.305655,493527019,684757099,493527019,2024-03-24,2024-03-24 00:00:00.000000 UTC,D,400.00,NGN,23387956.15,...,[70],[493527019],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-24 07:22:16, origina...",[transfers],3622512813388183/SUCCESS,3622512813388183
1,0.311698,493530399,684756545,493530399,2024-03-29,2024-03-29 00:00:00.000000 UTC,D,100.00,NGN,22723019.15,...,[70],[493530399],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-29 16:02:17, origina...",[transfers],4122389416478862/SUCCESS,4122389416478862
2,0.271384,493531321,684759010,493531321,2024-03-30,2024-03-30 00:00:00.000000 UTC,D,1000.00,NGN,17587584.15,...,[70],[493531321],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-30 12:59:53, origina...",[transfers],1332568146768148/SUCCESS,1332568146768148
3,0.275696,493523573,684760491,493523573,2024-03-17,2024-03-17 00:00:00.000000 UTC,D,1000.00,NGN,22078736.15,...,[70],[493523573],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-17 16:11:40, origina...",[transfers],7321192357487326/SUCCESS,7321192357487326
4,0.285613,493526877,684758199,493526877,2024-03-23,2024-03-23 00:00:00.000000 UTC,D,500.00,NGN,24417139.15,...,[70],[493526877],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-23 19:37:45, origina...",[transfers],2291774841227371/SUCCESS,2291774841227371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63826,0.162058,493532512,684757053,493532512,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12739233.15,...,[70],[493532511],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 19:43:11, origina...",[transfers],2729785645691423/SUCCESS,2729785645691423
63827,0.143900,493532514,684761613,493532514,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12724208.15,...,[70],[493532513],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 19:46:28, origina...",[transfers],8422621429232868/SUCCESS,8422621429232868
63828,0.143900,493532516,684761608,493532516,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25.00,NGN,12709183.15,...,[70],[493532515],NaN,NaN,NaN,NaN,"{transaction_date=2024-03-31 19:53:42, origina...",[transfers],3694364932936198/SUCCESS,3694364932936198
63829,-0.414399,493532517,684759440,493532517,2024-03-31,2024-03-31 00:00:00.000000 UTC,D,25320.70,NGN,12683862.45,...,NaN,NaN,NaN,NaN,NaN,NaN,"{transaction_date=2024-03-13 11:00:07, origina...","[others, unknown_pattern]",nan/SUCCESS,nan


In [36]:
cols = ['best_match_score','description_number_fsp','description_number','amount_fsp', 'amount_backend', 'transaction_date_fsp', 'transaction_date_backend']
cols

['best_match_score',
 'description_number_fsp',
 'description_number',
 'amount_fsp',
 'amount_backend',
 'transaction_date_fsp',
 'transaction_date_backend']

In [37]:
matched_results[cols].sort_values (by=['best_match_score'], ascending=False)

,best_match_score,description_number_fsp,description_number,amount_fsp,amount_backend,transaction_date_fsp,transaction_date_backend
27267,0.392442,3954677518531293,3954677518531293,7950.00,7950.00,2024-03-19,2024-03-19
10116,0.392442,7194835152357795,7194835152357795,73000.00,73000.00,2024-03-19,2024-03-19
5801,0.392442,6745435934499667,6745435934499667,270.00,270.00,2024-03-19,2024-03-19
10928,0.392442,4272979139137842,4272979139137842,99810.00,99810.00,2024-03-20,2024-03-20
10927,0.392442,1914399172658562,1914399172658562,29400.00,29400.00,2024-03-20,2024-03-20
...,...,...,...,...,...,...,...
38460,-0.243151,None,nan,10000000.00,1000.00,2024-03-18,2024-03-13
43329,-0.243151,None,nan,10000000.00,1000.00,2024-03-21,2024-03-13
56084,-0.250078,None,nan,20000000.00,1000.00,2024-03-28,2024-03-13
63899,-0.294737,None,6679932876218342,1899.05,1900.00,2024-03-31,2024-03-31


Set what % threshold of accuracy to see which holds best.

In [38]:
threshold = 0.392442 * 0.8

In [39]:
matched_results[cols].query(f"best_match_score >= {threshold}").sort_values (by=['best_match_score'], ascending=False)

,best_match_score,description_number_fsp,description_number,amount_fsp,amount_backend,transaction_date_fsp,transaction_date_backend
30708,0.392442,2556854842842774,2556854842842774,87000.00,87000.00,2024-03-20,2024-03-20
5801,0.392442,6745435934499667,6745435934499667,270.00,270.00,2024-03-19,2024-03-19
10927,0.392442,1914399172658562,1914399172658562,29400.00,29400.00,2024-03-20,2024-03-20
11104,0.392442,9891441686565163,9891441686565163,30300.00,30300.00,2024-03-20,2024-03-20
5799,0.392442,8168323776572639,8168323776572639,165000.00,165000.00,2024-03-20,2024-03-20
...,...,...,...,...,...,...,...
32287,0.313989,3476422197296839,3476422197296839,30000.00,30000.00,2024-03-31,2024-03-31
5686,0.313989,7165128628318245,7165128628318245,30000.00,30000.00,2024-03-31,2024-03-31
5681,0.313989,6629155816625273,6629155816625273,30000.00,30000.00,2024-03-28,2024-03-28
3396,0.313989,9772713881755554,9772713881755554,30000.00,30000.00,2024-03-28,2024-03-28


80 % matches 1720 entries.

In [43]:
matched_results[cols].query(f"best_match_score >= 0").sort_values (by=['best_match_score'], ascending=False)


,best_match_score,description_number_fsp,description_number,amount_fsp,amount_backend,transaction_date_fsp,transaction_date_backend
10116,0.392442,7194835152357795,7194835152357795,73000.00,73000.00,2024-03-19,2024-03-19
23808,0.392442,9221943625234538,9221943625234538,910.00,910.00,2024-03-20,2024-03-20
5801,0.392442,6745435934499667,6745435934499667,270.00,270.00,2024-03-19,2024-03-19
12648,0.392442,4486473658366536,4486473658366536,46900.00,46900.00,2024-03-20,2024-03-20
30708,0.392442,2556854842842774,2556854842842774,87000.00,87000.00,2024-03-20,2024-03-20
...,...,...,...,...,...,...,...
55606,0.004016,3678571544738625,3673362394135894,25.00,8000.00,2024-03-28,2024-03-28
58935,0.002722,8647756882838829,8729445794889834,25.00,450.00,2024-03-30,2024-03-30
49289,0.002544,8763259415738374,8773789919896269,25.00,2900.00,2024-03-25,2024-03-25
49189,0.002544,8792769845957891,8773789919896269,25.00,2900.00,2024-03-25,2024-03-25


10430 / 10630 matched.

98.1185 % match rate.
